In [1]:
import numpy as np
from deap import base, creator, tools, algorithms

# 定义目标函数
def objective_function(t, r, N, K, t0):
    term1 = r * t * N * (t - (N / K) * t)
    term2 = 1 / (1 + np.exp(-r * (t - t0)))
    return term1 + term2

# 定义适应度函数
def fitness_function(individual):
    t = individual[0]
    r = individual[1]
    N = individual[2]
    K = individual[3]
    t0 = individual[4]
    
    # 替换为实际的最小值和最大值
    min_val = 0
    max_val = 100
    
    # 计算原始目标函数值
    original_value = objective_function(t, r, N, K, t0)
    
    # 对目标函数值进行归一化处理
    normalized_value = (original_value - min_val) / (max_val - min_val)
    
    return normalized_value

# 遗传算法初始化
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 修改 ConsoleOutput 类
@engine.analysis_register
class ConsoleOutput(OnTheFlyAnalysis):
    master_only = True
    interval = 1
    
    def register_step(self, g, population, engine):
        # 从种群中获取最佳个体
        best_indv = tools.selBest(population, 1)[0]
        
        # 输出最佳适应度和个体信息
        msg = 'Generation: {}, best fitness: {:.3f}, Best Individual: {}'.format(g, best_indv.fitness.values[0], best_indv)
        engine.logger.info(msg)

# 运行遗传算法
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

engine = algorithms.eaMuPlusLambda(pop, toolbox, mu=100, lambda_=200, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
engine.run()

# 输出最终结果
best_indv = tools.selBest(population, 1)[0]
final_fitness = best_indv.fitness.values[0]
print("Final Best Fitness:", final_fitness)
print("Final Best Individual:", best_indv)


NameError: name 'engine' is not defined